## Treating a Function Like an Object

In [2]:
# Example 5-1: Create and test a function then read its __doc__ and check its type
def factorial(n):
    '''Return n!'''
    return 1 if n < 2 else n * factorial(n-1)

In [4]:
factorial(4)

24

In [5]:
factorial.__doc__

'Return n!'

In [6]:
type(factorial)

function

In [7]:
# Example 5-2: Use function through a different name, and pass function as argument
fact = factorial
fact

<function __main__.factorial>

In [8]:
fact(5)

120

In [9]:
map(factorial, range(10))

In [10]:
list(map(factorial, range(6)))

[1, 1, 2, 6, 24, 120]

## Higher-Ordered Functions

In [13]:
# Example 5-3: Sorting a list of words by length
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'respberry', 'banana']
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'respberry', 'strawberry']

In [15]:
# Example 5-4: Sorting a list of words by their reversed spelling
def reverse(word):
    return word[::-1]
reverse('testing')

'gnitset'

In [16]:
sorted(fruits, key=reverse)

['banana', 'apple', 'fig', 'respberry', 'strawberry', 'cherry']

## Modern Replacements for map, filter and reduce
Functional languages commonly offer the map, filter, and reduce higher-order func‐
tions (sometimes with different names). The map and filter functions are still builtins in Python 3, but since the introduction of list comprehensions and generator ex‐
pressions, they are not as important. A listcomp or a genexp does the job of map and
filter combined, but is more readable.

In [17]:
# Example 5-5
list(map(fact, range(6)))

[1, 1, 2, 6, 24, 120]

In [18]:
[fact(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [19]:
list(map(factorial, filter(lambda n: n % 2, range(6))))

[1, 6, 120]

In [20]:
[factorial(n) for n in range(6) if n % 2]

[1, 6, 120]

In [21]:
# Example 5-6
from functools import reduce
from operator import add

reduce(add, range(6))

15

In [23]:
sum(range(6))

15

## Anonymous Functions
The best use of anonymous functions is in the context of an argument list. 

In [25]:
# Example 5-7
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'respberry', 'banana']
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'respberry', 'strawberry', 'cherry']

## User-Defined Callable Types

In [28]:
# Example 5-8
import random

class BingoCage:
    
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        return self.pick()

In [29]:
bingo = BingoCage(range(3))
bingo.pick()

2

In [30]:
bingo()

1

In [32]:
callable(bingo)

True

## From Positional to Keyword-Only Parameters
One of the best features of Python functions is the extremely flexible parameter handling
mechanism, enhanced with keyword-only arguments in Python 3. Closely related are
the use of * and ** to “explode” iterables and mappings into separate arguments when
we call a function. To see these features in action, see the code for Example 5-10 and
tests showing its use in Example 5-11.

In [41]:
# Example 5-10
def tag(name, *content, cls=None, **attrs):
    '''Generate one or more HTML tags'''
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        print(attrs)
        attr_str = ''.join(' %s="%s"' % (attr, value) for attr, value in sorted(attrs.items()))
        print(attr_str)
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s<%s/' % (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [34]:
tag('br')

'<br />'

In [35]:
tag('p', 'hello')

'<p>hello<p/'

In [37]:
print(tag('p', 'hello', 'world'))

<p>hello<p/
<p>world<p/


In [38]:
tag('p', 'hello', id=33)

 id="33"


'<p id="33">hello<p/'

In [39]:
print(tag('p', 'hello', 'world', cls='sidebar'))

 class="sidebar"
<p class="sidebar">hello<p/
<p class="sidebar">world<p/


In [43]:
# Even the first positional argument can be passed as a keyword when tag is called
tag(content='testing', name="img")

{'content': 'testing'}
 content="testing"


'<img content="testing" />'

In [44]:
# Prefixing the my_tag dict with ** passes all its items as separate arguments, 
# which are then bound to the named parameters, with the remaining caught by **attrs.
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg', 'cls': 'framed'}
tag(**my_tag)

{'class': 'framed', 'src': 'sunset.jpg', 'title': 'Sunset Boulevard'}
 class="framed" src="sunset.jpg" title="Sunset Boulevard"


'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

In [45]:
tag(my_tag)

"<{'cls': 'framed', 'src': 'sunset.jpg', 'title': 'Sunset Boulevard', 'name': 'img'} />"